220. Contains Duplicate III
Medium

1764

1827

Add to List

Share
Given an integer array nums and two integers k and t, return true if there are two distinct indices i and j in the array such that abs(nums[i] - nums[j]) <= t and abs(i - j) <= k.

 

Example 1:

Input: nums = [1,2,3,1], k = 3, t = 0
Output: true
Example 2:

Input: nums = [1,0,1,1], k = 1, t = 2
Output: true
Example 3:

Input: nums = [1,5,9,1,5,9], k = 2, t = 3
Output: false
 

Constraints:

0 <= nums.length <= 2 * 104
-231 <= nums[i] <= 231 - 1
0 <= k <= 104
0 <= t <= 231 - 1

In [ ]:
class Solution:
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        """hash table - TLE"""
        # dic = {}
        # for i, num in enumerate(nums):
        #     for key in dic.keys():
        #         if key[0] <= num <= key[1] and i - dic[key] <= k:
        #             return True
        #     dic[(num-t, num+t)] = i
        # return False
        
        """set of fixed size - TLE"""
        # seen = set()
        # for i, num in enumerate(nums):
        #     if num in seen:
        #         return True
        #     for j in range(num-t, num+t+1):
        #         seen.add(j)
        #     # remove the oldest element
        #     if i >= k:
        #         for j in range(nums[i-k]-t, nums[i-k]+t+1):
        #             seen.remove(j)
        # return False

In [ ]:
class Solution:
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        """
        sliding window + bucket sort: O(n)
        assign each num to a bucket of size (t+1)
        (1) if the new num falls into the same bucket - duplicate found
        (2) if the new num is assigned to a new bucket next to an existing bucket - check distance 
        (3) else: the distance must be grater than t - requirement not met
        sliding:
        remove the bucket for the left most num, as there is only one num per bucket 
        """
        if k < 1 or len(nums) < 2:
            return False
        cache = {}
        for i in range(len(nums)):
            # sliding at the begining of each iteration - remove the bucket out of range
            if i > k:
                cache.pop(nums[i-k-1]//(t+1))
                
            bucket = nums[i]//(t+1)
            if bucket in cache:
                return True
            if bucket - 1 in cache and nums[i] - cache[bucket-1] <= t:
                return True
            if bucket + 1 in cache and cache[bucket+1] - nums[i] <= t:
                return True
            # no duplicate found, add the new bucket
            cache[bucket] = nums[i]
            
            # or sliding at the end of each iteration - remove the bucket out of range
#             if i >= k:
#                 cache.pop(nums[i-k]//(t+1))
        return False     

In [ ]:
class Solution:
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        """OrderedDict: similar idea of assigning num to different buckets"""
        if k < 1 or len(nums) < 2:
            return False
        from collections import OrderedDict # dict is ordered, not sorted
        window = OrderedDict() # key: bucket - value: num
        for num in nums:
            # sliding
            if len(window) > k:
                window.popitem(last=False) # popleft: remove the oldest pair
            bucket = num if not t else num//t
            # if key not in dic: dic.get(key) return None
            # m is the corresponding value in the bucket or neighboring bucket
            for m in (window.get(bucket-1), window.get(bucket), window.get(bucket+1)):
                # must check if m is not None because m can be 0
                if m is not None and abs(num - m) <= t: # check difference if m is found in the OrderedDict
                    return True
            window[bucket] = num
        return False 

In [ ]:
from sortedcontainers import SortedList
class Solution:
    def containsNearbyAlmostDuplicate(self, nums: List[int], k: int, t: int) -> bool:
        """
        SortedList: O(nlogk) - n steps: in each step logk for binary search
        
        bisect_left(val) returns the index of val in the list if it were inserted at the leftmost position in sorted order. 
        This is the same as the number of elements strictly less than val. (called a rank query on the binary search Wikipedia page)

        bisect_right(val) returns the index of val in the list if it were inserted at the rightmost position in sorted order. 
        This is the same as the number of elements less than or equal to val.
        
        bisect_right(val)-bisect_left(val) results in the number of val in the list
        """
        
        sl = SortedList()
        for i in range(len(nums)):
            if i > k:
                sl.remove(nums[i-k-1])
            left = SortedList.bisect_left(sl, nums[i] - t)
            right = SortedList.bisect_right(sl, nums[i] + t)
            # right - left is the number of element within [nums[i]-t, nums[i]+t]in SortedList
            if right > left:
                return True
            sl.add(nums[i])
        return False